In [9]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.metrics import accuracy_score
import numpy
from sklearn.metrics import mean_squared_error
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split


In [10]:
vdata = pd.read_csv("VariantTrainingData.csv")
vdata.head(10)

vdata.loc[vdata["Classified"] == 'VOI', "Classified"] = 0
vdata.loc[vdata["Classified"] == 'VOC', "Classified"] = 1
vdata.loc[vdata["Classified"] == 'VUM', "Classified"] = 2

spikeP = vdata["AA Substitutions"]
lineage = vdata["Lineage"]
clade = vdata["Clade"]
variant = vdata["Classified"]

vdata.head(10)


,Virus name,Accession ID,Collection date,Location,Host,Passage,Specimen,Additional host information,Sequencing technology,Assembly method,Comment,Comment type,Lineage,Clade,AA Substitutions,Variant,Classified
0,hCoV-19/England/CAMC-149B04F/2021,EPI_ISL_1483874,3/30/2021,Europe / United Kingdom / England,Human,Original,NaN,NaN,Illumina NovaSeq,NaN,Gap of 30 nucleotides when compared to the ref...,warn,C.37,GR,"(Spike_L249del,Spike_S247del,Spike_G75V,Spike_...",VOI Lambda GR/452Q.V1 (C.37+C.37.1) first dete...,0
1,hCoV-19/USA/NY-CDC-QDX23179684/2021,EPI_ISL_1491965,3/20/2021,North America / USA / New York,Human,Original,Nasal swab,NaN,Illumina MiSeq,BWA v.7.12,Gap of 30 nucleotides when compared to the ref...,warn,C.37,GR,"(Spike_L249del,Spike_S247del,Spike_G75V,Spike_...",VOI Lambda GR/452Q.V1 (C.37+C.37.1) first dete...,0
2,hCoV-19/USA/CA-CDC-FG-010085/2021,EPI_ISL_1481945,3/13/2021,North America / USA / California,Human,Original,Nasal swab,NaN,Illumina NovaSeq,BWA; iVar 1.3,Gap of 30 nucleotides when compared to the ref...,warn,C.37,GR,"(Spike_L249del,Spike_S247del,Spike_G75V,Spike_...",VOI Lambda GR/452Q.V1 (C.37+C.37.1) first dete...,0
3,hCoV-19/USA/TN-CDC-ASC210017485/2021,EPI_ISL_1491924,3/24/2021,North America / USA / Tennessee,Human,Original,Nasal - Anterior Nares,NaN,Illumina NovaSeq 6000,Dragen COVID Lineage v3.5.1,Gap of 30 nucleotides when compared to the ref...,warn,C.37,GR,"(Spike_L249del,Spike_S247del,Spike_G75V,Spike_...",VOI Lambda GR/452Q.V1 (C.37+C.37.1) first dete...,0
4,hCoV-19/USA/TN-CDC-ASC210016026/2021,EPI_ISL_1513430,3/23/2021,North America / USA / Tennessee,Human,Original,Nasal swab,NaN,Illumina NovaSeq 6000,Dragen COVID Lineage v3.5.1,Gap of 30 nucleotides when compared to the ref...,warn,C.37,GR,"(Spike_L249del,Spike_S247del,Spike_G75V,Spike_...",VOI Lambda GR/452Q.V1 (C.37+C.37.1) first dete...,0
5,hCoV-19/USA/TN-CDC-ASC210016009/2021,EPI_ISL_1513424,3/23/2021,North America / USA / Tennessee,Human,Original,Nasopharyngeal swab,NaN,Illumina NovaSeq 6000,Dragen COVID Lineage v3.5.1,Gap of 30 nucleotides when compared to the ref...,warn,C.37,GR,"(Spike_L249del,Spike_S247del,Spike_G75V,Spike_...",VOI Lambda GR/452Q.V1 (C.37+C.37.1) first dete...,0
6,hCoV-19/USA/AR-CDC-LC0035111/2021,EPI_ISL_1515553,3/23/2021,North America / USA / Arkansas,Human,Original,Nasal swab,NaN,PacBio Sequel II,CLC Genomics,Long stretches of NNNs (6.74% of overall seque...,alert,C.37,GR,"(N_T366I,N_G214C,NSP5_G15S,NSP6_G107del,NSP6_S...",VOI Lambda GR/452Q.V1 (C.37+C.37.1) first dete...,0
7,hCoV-19/USA/AZ-CDC-LC0035336/2021,EPI_ISL_1515705,3/26/2021,North America / USA / Arizona,Human,Original,Nasal swab,NaN,PacBio Sequel II,CLC Genomics,Gap of 30 nucleotides when compared to the ref...,warn,C.37,GR,"(Spike_L249del,Spike_S247del,Spike_G75V,Spike_...",VOI Lambda GR/452Q.V1 (C.37+C.37.1) first dete...,0
8,hCoV-19/Canada/QC-1nIUH-5524359940/2021,EPI_ISL_2975479,4/5/2021,North America / Canada / Quebec,Human,Original,NaN,NaN,Illumina_NexteraFlex,iVar 1.3,Gap of 9 nucleotides when compared to the refe...,info,C.37,GR,"(Spike_G75V,NSP6_S106del,N_R203K,Spike_F490S,S...",VOI Lambda GR/452Q.V1 (C.37+C.37.1) first dete...,0
9,hCoV-19/USA/NY-MSHSPSP-PV35341/2021,EPI_ISL_1709289,3/10/2021,North America / USA / New York / Queens County,Human,Original,NaN,NaN,Illumina MiSeq,"Minimap2, Pilon",Gap of 30 nucleotides when compared to the ref...,warn,C.37,GR,"(Spike_L249del,Spike_S247del,Spike_G75V,Spike_...",VOI Lambda GR/452Q.V1 (C.37+C.37.1) first dete...,0


In [11]:
spikeP

0       (Spike_L249del,Spike_S247del,Spike_G75V,Spike_...
1       (Spike_L249del,Spike_S247del,Spike_G75V,Spike_...
2       (Spike_L249del,Spike_S247del,Spike_G75V,Spike_...
3       (Spike_L249del,Spike_S247del,Spike_G75V,Spike_...
4       (Spike_L249del,Spike_S247del,Spike_G75V,Spike_...
                              ...                        
3995    (Spike_T95I,Spike_E484K,NSP7_S26F,NS3_L106F,NS...
3996    (NSP13_P419S,NSP6_Q160R,Spike_T95I,N_A152G,Spi...
3997    (NSP13_P419S,NSP12_M629I,Spike_T95I,Spike_D950...
3998    (Spike_T95I,NSP1_M85del,Spike_E484K,NSP14_L177...
3999    (Spike_T95I,NSP1_M85del,Spike_E484K,NSP14_L177...
Name: AA Substitutions, Length: 4000, dtype: object

In [12]:
cv=CountVectorizer()

In [45]:
x_train, x_test, y_train, y_test = train_test_split(spikeP, variant, test_size=0.2)

x_traincv = cv.fit_transform(x_train)
x_traincv.toarray()
print(cv.get_feature_names_out())
print(y_train.astype('int'))

['e_a36v' 'e_d72g' 'e_d72y' ... 'spike_y495h' 'spike_y505h' 'spike_y660h']
2052    1
2953    2
1396    1
3603    0
2447    1
       ..
3479    1
3014    1
1608    1
2053    1
2400    1
Name: Classified, Length: 3200, dtype: int32


In [52]:
multiNom = MultinomialNB()
multiNom.fit(x_traincv, y_train.astype('int'))

MultinomialNB()

In [55]:
x_testcv = cv.transform(x_test)
print(multiNom.score(x_testcv, y_test.astype('int')))

x_predict = multiNom.predict(x_testcv)
print(accuracy_score(x_predict, y_test.astype('int')))

0.99875
0.99875


In [56]:
GausNB = GaussianNB()
GausNB.fit(x_traincv.toarray(),y_train.astype('int'))

print(GausNB.score(x_testcv.toarray(),y_test.astype('int')))


0.9975
